In [22]:
from math import ceil

import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import DenseGraphConv, GCNConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch

import sys
import numpy as np

sys.path.append("../")
import utils
import torch_geometric as tg

from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, accuracy_score

In [23]:
early_stop_thresh = 25
best_macro_f1 = -1


train_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TRAIN", True, n_quantiles=100
)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

test_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TEST", True, n_quantiles=100
)
test_loader = DataLoader(test_dataset, batch_size=64)

In [24]:
class Net(torch.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        graph_sizes,
        n_edge_layers=2,
        hidden_channels=32,
    ):
        super().__init__()
        self.edge_layers = torch.nn.ModuleList()
        for it in range(n_edge_layers):
            nn = tg.nn.MLP([in_channels, hidden_channels, hidden_channels])
            self.edge_layers.append(tg.nn.GINConv(nn))
            in_channels = hidden_channels
        self.gcn_layers = torch.nn.ModuleList()
        for n_nodes in graph_sizes:
            self.gcn_layers.append(Linear(hidden_channels, n_nodes))
            nn = tg.nn.MLP([hidden_channels, hidden_channels])
            self.gcn_layers.append(tg.nn.DenseGINConv(nn))
            nn = tg.nn.MLP([hidden_channels, hidden_channels])
            self.gcn_layers.append(tg.nn.DenseGINConv(nn))
        nn = tg.nn.MLP([hidden_channels, hidden_channels])
        self.gcn_layers.append(tg.nn.DenseGINConv(nn))

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr, batch = (
            data.x,
            data.edge_index,
            data.edge_attr,
            data.batch,
        )
        for layer in self.edge_layers:
            x = layer(x, edge_index).relu()
        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)

        mincut_loss = torch.zeros(1)
        ortho_loss = torch.zeros(1)
        for layer in self.gcn_layers:
            if layer._get_name() == "Linear":
                s = layer(x)
                x, adj, mc, o = dense_mincut_pool(x, adj, s, mask)
                mincut_loss += mc
                ortho_loss += o
                mask = None
            else:
                x = layer(x, adj).relu()
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mincut_loss, ortho_loss

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(1, 5, [50, 20], hidden_channels=8).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min", cooldown=2, factor=0.5, verbose=True
)

In [28]:
# def train():
#     model.train()

#     total_loss = 0
#     for data in train_loader:
#         optimizer.zero_grad()
#         y_out,mc_loss, o_loss  = model(data)
#         loss = F.nll_loss(y_out, data.y)+mc_loss+o_loss
#         # print(F.nll_loss(y_out, data.y),"and",mc_loss+o_loss)
#         loss.backward()
#         optimizer.step()
#         total_loss += float(loss) * data.num_graphs
#     return total_loss / len(train_loader.dataset)


# @torch.no_grad()
# def test(loader):
#     model.eval()
#     y_pred = []
#     y_true = []
#     loss = 0
#     for data in loader:
#         y_out,mc_loss, o_loss = model(data)
#         y_pred.append(y_out.argmax(dim=-1))
#         y_true.append(data.y - 1)
#         loss += float(F.nll_loss(y_out, data.y) * data.num_graphs + mc_loss + o_loss)
#     y_pred = np.concatenate(y_pred)
#     y_true = np.concatenate(y_true)
#     return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), accuracy_score(y_true=y_true,y_pred=y_pred), loss / len(
#             loader.dataset
#         )


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, mc_loss, o_loss = model(data)
        loss = F.nll_loss(out, data.y.view(-1)) + mc_loss + o_loss
        loss.backward()
        loss_all += data.y.size(0) * float(loss)
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0

    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = model(data)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        loss_all += data.y.size(0) * float(loss)
        correct += int(pred.max(dim=1)[1].eq(data.y.view(-1)).sum())

    return loss_all / len(loader.dataset), correct / len(loader.dataset)


best_val_acc = test_acc = 0
best_val_loss = float("inf")
patience = start_patience = 50
for epoch in range(1, 2000):
    train_loss = train(epoch)
    _, train_acc = test(train_loader)
    val_loss, val_acc = test(test_loader)
    if val_loss < best_val_loss:
        test_loss, test_acc = test(test_loader)
        best_val_acc = val_acc
        patience = start_patience
    else:
        patience -= 1
        if patience == 0:
            break
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )

Epoch: 001, Train_Loss: 2.0645,Test_Loss: 2.1069,Train_acc: 0.3000,Test_acc: 0.2169
Epoch: 002, Train_Loss: 2.0597,Test_Loss: 2.1024,Train_acc: 0.3100,Test_acc: 0.2227
Epoch: 003, Train_Loss: 2.0552,Test_Loss: 2.0973,Train_acc: 0.3000,Test_acc: 0.2289
Epoch: 004, Train_Loss: 2.0512,Test_Loss: 2.0920,Train_acc: 0.3100,Test_acc: 0.2326
Epoch: 005, Train_Loss: 2.0470,Test_Loss: 2.0866,Train_acc: 0.3300,Test_acc: 0.2346
Epoch: 006, Train_Loss: 2.0424,Test_Loss: 2.0808,Train_acc: 0.3200,Test_acc: 0.2326
Epoch: 007, Train_Loss: 2.0375,Test_Loss: 2.0751,Train_acc: 0.3300,Test_acc: 0.2713
Epoch: 008, Train_Loss: 2.0325,Test_Loss: 2.0697,Train_acc: 0.3300,Test_acc: 0.3419
Epoch: 009, Train_Loss: 2.0276,Test_Loss: 2.0644,Train_acc: 0.3900,Test_acc: 0.3856
Epoch: 010, Train_Loss: 2.0224,Test_Loss: 2.0586,Train_acc: 0.4300,Test_acc: 0.4276
Epoch: 011, Train_Loss: 2.0168,Test_Loss: 2.0523,Train_acc: 0.4400,Test_acc: 0.4503
Epoch: 012, Train_Loss: 2.0109,Test_Loss: 2.0464,Train_acc: 0.4700,Test_acc:

KeyboardInterrupt: 

In [27]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    # elif epoch - best_epoch > early_stop_thresh:
    #     print(
    #         f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
    #     )
    #     break

Epoch: 001, Train_Loss: 1.6379,Test_Loss: 1.6328,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 002, Train_Loss: 1.6367,Test_Loss: 1.6324,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 003, Train_Loss: 1.6358,Test_Loss: 1.6322,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 004, Train_Loss: 1.6348,Test_Loss: 1.6317,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 005, Train_Loss: 1.6342,Test_Loss: 1.6312,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 006, Train_Loss: 1.6336,Test_Loss: 1.6308,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 007, Train_Loss: 1.6332,Test_Loss: 1.6305,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 008, Train_Loss: 1.6328,Test_Loss: 1.6301,Train_f1: 0.0667,Test_f1: 0.0459,Train_acc: 0.2000,Test_acc: 0.1295
Epoch: 009, Train_Loss: 1.6324,Test_Loss: 1.6298,Train_f1: 0.066

KeyboardInterrupt: 